## Package

In [13]:
import pandas as pd
import numpy as np
from datasets import load_dataset
from huggingface_hub import login
import json
import os
import time
from datetime import datetime
from tqdm import tqdm
import re
import glob
import ast 

## Get Data

In [ ]:
# 資料讀取選項
# 您可以選擇以下任一種方式來載入資料：

# 選項 1: 從已儲存的本地檔案讀取 (推薦，速度快)
use_local_files = False

# 選項 2: 從 Hugging Face 直接串流載入 (需要網路連線)
use_streaming = False

# 選項 3: 下載完整資料集 (檔案很大，不推薦)
use_full_download = True

print("=== 資料載入選項 ===")
print(f"使用本地檔案: {use_local_files}")
print(f"使用串流模式: {use_streaming}")
print(f"下載完整資料集: {use_full_download}")

# 資料載入
if use_local_files:
    print("\n📁 從本地檔案讀取資料...")
    
    # 檢查已儲存的檔案
    save_dir = "saved_datasets"
    
    if os.path.exists(save_dir):
        import glob
        
        # 尋找可用的檔案
        csv_files = glob.glob(f"{save_dir}/*.csv")
        json_files = glob.glob(f"{save_dir}/*.json")
        parquet_files = glob.glob(f"{save_dir}/*.parquet")
        
        print(f"找到的檔案:")
        print(f"  CSV 檔案: {len(csv_files)} 個")
        print(f"  JSON 檔案: {len(json_files)} 個")
        print(f"  Parquet 檔案: {len(parquet_files)} 個")
        
        # 優先使用 Parquet 檔案 (最高效)
        if parquet_files:
            latest_file = max(parquet_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 Parquet 檔案: {latest_file}")
            df = pd.read_parquet(latest_file)
            
        # 其次使用 CSV 檔案
        elif csv_files:
            latest_file = max(csv_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 CSV 檔案: {latest_file}")
            df = pd.read_csv(latest_file)
            
        # 最後使用 JSON 檔案
        elif json_files:
            latest_file = max(json_files, key=os.path.getctime)
            print(f"\n📊 讀取最新的 JSON 檔案: {latest_file}")
            with open(latest_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            df = pd.DataFrame(data)
            
        else:
            print("❌ 沒有找到已儲存的資料檔案")
            print("請先執行資料下載和儲存的程式碼")
            df = None
    else:
        print("❌ 找不到 saved_datasets 目錄")
        print("請先執行資料下載和儲存的程式碼")
        df = None

elif use_streaming:
    print("\n🌐 從 Hugging Face 串流載入資料...")
    
    # 使用串流模式載入資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset", streaming=True)
    
    # 設定要載入的樣本數量 - 減少到100筆用於演示
    num_samples = 1000
    print(f"載入前 {num_samples} 筆資料...")
    
    # 收集資料
    sample_data = []
    for i, example in enumerate(dataset['train']):
        if i >= num_samples:
            break
        sample_data.append(example)
        if (i + 1) % 25 == 0:
            print(f"  已載入 {i + 1} 筆資料...")
    
    # 轉換為 DataFrame
    df = pd.DataFrame(sample_data)
    
elif use_full_download:
    print("\n⬇️ 下載完整資料集...")
    print("警告：這將下載 13.7GB 的資料，可能需要很長時間")
    
    # 下載完整資料集
    dataset = load_dataset("austenjs/ClueCorpusSmallDataset")
    df = dataset['train'].to_pandas()

else:
    print("❌ 沒有選擇任何資料載入選項")
    df = None

# 顯示資料資訊
if df is not None:
    print(f"\n✅ 資料載入成功！")
    print(f"📊 資料形狀: {df.shape}")
    print(f"📋 欄位名稱: {list(df.columns)}")
    
    # 顯示基本統計
    if 'text' in df.columns: # type: ignore
        df['text_length'] = df['text'].str.len() # type: ignore
        print(f"\n📈 文本長度統計:")
        print(df['text_length'].describe()) # type: ignore
        
        # 顯示前幾筆資料範例
        print(f"\n📝 前 3 筆資料範例:")
        for i in range(min(3, len(df))): # type: ignore
            text = df.iloc[i]['text']
            # 顯示前100個字符
            preview = text[:100] + "..." if len(text) > 100 else text
            print(f"範例 {i+1} ({len(text)} 字符): {preview}")
            print("-" * 80)
    
    print(f"\n🎯 資料已準備就緒，可用於後續的 LLM 評分處理！")
else:
    print("\n❌ 資料載入失敗，請檢查設定並重新執行")

# 儲存到全域變數供後續使用
globals()['dataset_df'] = df

=== 資料載入選項 ===
使用本地檔案: False
使用串流模式: False
下載完整資料集: True

⬇️ 下載完整資料集...
警告：這將下載 13.7GB 的資料，可能需要很長時間


Loading dataset shards:   0%|          | 0/28 [00:00<?, ?it/s]

In [ ]:

save_dir = "saved_datasets"
os.makedirs(save_dir, exist_ok=True)
df.to_csv(os.path.join(save_dir, "test_cluecorpus.csv"), index=False, encoding="utf-8-sig")


In [ ]:
# import pandas as pd

# dataset_df = pd.read_csv("test_cluecorpus.csv", encoding="utf-8-sig")

In [4]:
dataset_df = pd.read_csv(
    "test_cluecorpus.csv",
    encoding="utf-8-sig",
    nrows=1000   # 只讀取前 1000 筆
)

print(len(dataset_df))

1000


## 📝 文本切分處理

In [5]:
import os
import json
import pandas as pd
from datetime import datetime
from tqdm import tqdm

print("🔪 啟動文本切分處理 (含斷點續存)...")

# =========================================================
# 句子切分核心函數
# =========================================================
def split_text_to_sentences(text, min_length=30, max_length=250):
    sentence_separators = ['。', '！', '？', '；', '…']  # 強分隔
    phrase_separators = ['，', '、', '：', '；']       # 弱分隔

    sentences, current_sentence = [], ""
    for char in text:
        current_sentence += char
        if char in sentence_separators:
            if current_sentence.strip():
                sentences.append(current_sentence.strip())
                current_sentence = ""
    if current_sentence.strip():
        sentences.append(current_sentence.strip())

    fragments = []
    for sentence in sentences:
        if len(sentence) < min_length:
            continue
        if len(sentence) <= max_length:
            fragments.append(sentence)
        else:
            # 長句再細切
            parts, current_part = [], ""
            for char in sentence:
                current_part += char
                if char in phrase_separators:
                    if current_part.strip() and len(current_part.strip()) >= min_length:
                        parts.append(current_part.strip())
                        current_part = ""
            if current_part.strip() and len(current_part.strip()) >= min_length:
                parts.append(current_part.strip())

            merged_parts, temp_part = [], ""
            for part in parts:
                if len(temp_part + part) <= max_length:
                    temp_part = temp_part + part if temp_part else part
                else:
                    if temp_part and len(temp_part) >= min_length:
                        merged_parts.append(temp_part)
                    temp_part = part
            if temp_part and len(temp_part) >= min_length:
                merged_parts.append(temp_part)

            # 無法細切就硬切
            if not merged_parts and len(sentence) >= min_length:
                for i in range(0, len(sentence), max_length):
                    fragment = sentence[i:i + max_length]
                    if len(fragment) >= min_length:
                        merged_parts.append(fragment)

            fragments.extend(merged_parts)
    return fragments

def split_text_by_punctuation(text, min_length=30, max_length=250):
    return split_text_to_sentences(text, min_length, max_length)


# =========================================================
# 斷點續存輔助函數
# =========================================================
CHECKPOINT_FILE = "split_checkpoint.json"
PARTIAL_FILE    = "split_partial.csv"

def save_checkpoint(last_index):
    with open(CHECKPOINT_FILE, "w", encoding="utf-8") as f:
        json.dump({"last_index": last_index}, f)
    # print(f"💾 已儲存進度：index {last_index}")

def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
            ckpt = json.load(f)
        print(f"🔄 偵測到進度檔，將從 index {ckpt['last_index'] + 1} 繼續")
        return ckpt["last_index"]
    return -1


# =========================================================
# 主處理流程（已加入斷點續存）
# =========================================================
def process_text_splitting(df, text_column='text', min_length=30, max_length=250):
    print(f"📊 開始處理文本切分...")
    print(f"  原始資料筆數: {len(df)}")
    print(f"  句子片段長度範圍: {min_length}-{max_length} 字")

    last_index = load_checkpoint()
    split_data = []

    # 讀取部分輸出以便續存
    if os.path.exists(PARTIAL_FILE):
        split_data = pd.read_csv(PARTIAL_FILE).to_dict("records")
        print(f"🔁 已讀取部分處理結果：{len(split_data)} 筆")

    total_fragments, processed_texts = len(split_data), 0

    CHECKPOINT_INTERVAL = 1000

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="切分進度"):
        if idx <= last_index:
            continue  # 跳過已處理

        if idx % CHECKPOINT_INTERVAL == 0:
            save_checkpoint(idx)
            pd.DataFrame(split_data).to_csv(PARTIAL_FILE, index=False, encoding="utf-8-sig")

        original_text = row[text_column]
        if pd.isna(original_text):
            continue          # ⬅️ 建議直接跳過
        original_text = str(original_text)
        
        if len(original_text) < min_length:
            continue

        search_start_pos = 0
        fragments = split_text_to_sentences(original_text, min_length, max_length)

        for frag_idx, fragment in enumerate(fragments):
            start_pos = original_text.find(fragment, search_start_pos)
            end_pos   = start_pos + len(fragment) if start_pos != -1 else -1
            if start_pos != -1:
                search_start_pos = end_pos

            new_row = row.copy()
            new_row[text_column] = fragment
            new_row['original_index'] = idx
            new_row['fragment_index'] = frag_idx
            new_row['original_text_length'] = len(original_text)
            new_row['fragment_length'] = len(fragment)
            new_row['source_type'] = 'sentence_fragment'
            new_row['fragment_start'] = start_pos
            new_row['fragment_end'] = end_pos
            split_data.append(new_row)
            total_fragments += 1

        processed_texts += 1

        # ✅ 每處理一筆就存檔
        save_checkpoint(idx)
        pd.DataFrame(split_data).to_csv(PARTIAL_FILE, index=False, encoding="utf-8-sig")

    # 全部完成後刪除 checkpoint
    if os.path.exists(CHECKPOINT_FILE):
        os.remove(CHECKPOINT_FILE)
        print("🎉 全部處理完成，已刪除斷點檔")

    split_df = pd.DataFrame(split_data)

    print(f"\n✅ 文本切分完成！")
    print(f"  處理文本數: {processed_texts}")
    print(f"  生成句子片段數: {total_fragments}")
    if processed_texts:
        print(f"  平均每文本片段數: {total_fragments/processed_texts:.1f}")

    if not split_df.empty:
        length_stats = split_df['fragment_length'].describe()
        print(f"\n📏 片段長度統計:")
        print(f"  平均長度: {length_stats['mean']:.1f} 字")
        print(f"  最短片段: {length_stats['min']:.0f} 字")
        print(f"  最長片段: {length_stats['max']:.0f} 字")
        print(f"  中位數長度: {length_stats['50%']:.1f} 字")

    return split_df


# =========================================================
# 執行流程示範
# =========================================================
if 'dataset_df' in globals() and dataset_df is not None:
    MIN_FRAGMENT_LENGTH = 30
    MAX_FRAGMENT_LENGTH = 250

    print(f"\n⚙️ 切分參數:")
    print(f"  最小片段長度: {MIN_FRAGMENT_LENGTH} 字")
    print(f"  最大片段長度: {MAX_FRAGMENT_LENGTH} 字")
    print(f"  切分模式: 句子級別 + 斷點續存")

    split_dataset_df = process_text_splitting(
        df=dataset_df,
        text_column='text',
        min_length=MIN_FRAGMENT_LENGTH,
        max_length=MAX_FRAGMENT_LENGTH
    )

    if not split_dataset_df.empty:
        # 另存最終檔案（附時間戳）
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        split_dir = "split_datasets"
        os.makedirs(split_dir, exist_ok=True)

        base_filename = f"{split_dir}/sentence_fragments_{timestamp}"
        csv_filename = f"{base_filename}.csv"
        json_filename = f"{base_filename}.json"
        parquet_filename = f"{base_filename}.parquet"

        int_cols = [
        "original_index", "fragment_index",
        "original_text_length", "fragment_length",
        "fragment_start", "fragment_end"
        ]
        for c in int_cols:
            if c in split_dataset_df.columns:
                split_dataset_df[c] = (
                    pd.to_numeric(split_dataset_df[c], errors="coerce")
                    .astype("Int64")
                )

        split_dataset_df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
        split_dataset_df.to_json(json_filename, orient='records', force_ascii=False, indent=2)
        split_dataset_df.to_parquet(parquet_filename, index=False)

        print(f"\n💾 完成輸出:")
        print(f"  📄 CSV: {csv_filename}")
        print(f"  📋 JSON: {json_filename}")
        print(f"  📦 Parquet: {parquet_filename}")

        globals()['split_dataset_df'] = split_dataset_df
        print(f"\n🎯 變數名稱: split_dataset_df")
    else:
        print("❌ 沒有產生有效片段")
        split_dataset_df = None
else:
    print("❌ 沒有找到資料集，請先執行 Get Data")
    split_dataset_df = None

print("=" * 80)


🔪 啟動文本切分處理 (含斷點續存)...

⚙️ 切分參數:
  最小片段長度: 30 字
  最大片段長度: 250 字
  切分模式: 句子級別 + 斷點續存
📊 開始處理文本切分...
  原始資料筆數: 1000
  句子片段長度範圍: 30-250 字
🔁 已讀取部分處理結果：32800 筆


切分進度: 100%|██████████| 1000/1000 [02:02<00:00,  8.19it/s]


🎉 全部處理完成，已刪除斷點檔

✅ 文本切分完成！
  處理文本數: 794
  生成句子片段數: 36670
  平均每文本片段數: 46.2

📏 片段長度統計:
  平均長度: 62.0 字
  最短片段: 30 字
  最長片段: 1049 字
  中位數長度: 51.0 字

💾 完成輸出:
  📄 CSV: split_datasets/sentence_fragments_20250911_201937.csv
  📋 JSON: split_datasets/sentence_fragments_20250911_201937.json
  📦 Parquet: split_datasets/sentence_fragments_20250911_201937.parquet

🎯 變數名稱: split_dataset_df


In [6]:
 split_dataset_df.head(5)

,text,text_length,original_index,fragment_index,original_text_length,fragment_length,source_type,fragment_start,fragment_end
0,兰州公交集团:明起至8月底三条公交线暂不经过靖远路站原标题：明起至8月底三条公交线暂不经过靖...,545,3,0,545,155,sentence_fragment,0,155
1,据悉，35路公交线由兰州西客站开往美伦广场时，兰州西客站至白塔山公园线路不变，经北滨河路九州...,545,3,1,545,71,sentence_fragment,155,226
2,由美伦广场开往兰州西客站时，美伦广场至庙滩子线路不变，左转弯进入九州大道至北滨河路市二医院恢...,545,3,2,545,53,sentence_fragment,226,279
3,81路公交线路由市二医院开往省军干所时，经北滨河路九州大道南口左转弯进入九州大道至庙滩子恢复...,545,3,3,545,52,sentence_fragment,279,331
4,由省军干所开往市二医院时，省军干所至庙滩子线路不变，左转弯进入九州大道至市二医院。,545,3,4,545,41,sentence_fragment,331,372


## LLM AUG

In [15]:
# # ==== CONFIG ====
# MODEL_API_ENDPOINT = "https://labor- openwebui.dgx-coolify.apmic.ai/api/"
# API_KEY  = "請填入自己的API KEY"
# MODEL_NAME  = "gemma-3-27b-it"
# BATCH_SIZE  = 20          # 視API能力可調
# THRESHOLD   = 3
# OUTPUT_DIR  = "./results" # 儲存結果的資料夾

In [ ]:
API_KEY ="sk-b56c488f33b94df297a6314bd037b805"

## gemma- 100

In [11]:
import nest_asyncio
nest_asyncio.apply()
import aiohttp
import asyncio

CHECKPOINT_FILE = "mainland_checkpoint.json"
PARTIAL_FILE    = "mainland_partial.csv"

def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, "r", encoding="utf-8") as f:
            ckpt = json.load(f)
        print(f"🔄 偵測到進度檔，將從 index {ckpt['last_index'] + 1} 繼續")
        return ckpt["last_index"]
    return -1

def save_checkpoint(last_index):
    with open(CHECKPOINT_FILE, "w", encoding="utf-8") as f:
        json.dump({"last_index": last_index}, f)


# 🎯 最終版大陸用語識別與篩選系統 - 使用 Ollama 推論並儲存結果
print("🚀 啟動最終版大陸用語識別系統...")

# 定義大陸特有詞彙庫
mainland_terms = {
    "計算機": ["電腦"], "軟件": ["軟體"], "硬件": ["硬體"], "網絡": ["網路"], 
    "數據": ["資料"], "程序": ["程式"], "信息": ["資訊"], "出租車": ["計程車"],
    "公交車": ["公車"], "地鐵": ["捷運"], "質量": ["品質"], "服務員": ["服務生"],
    "土豆": ["馬鈴薯"], "西紅柿": ["番茄"], "搞定": ["完成"], "挺": ["很"],
    "咋": ["怎麼"], "啥": ["什麼"], "微信": [""], "支付寶": [""], "淘寶": [""]
}

# 大陸語法模式
mainland_patterns = [r"挺.*的", r"蠻.*的", r".*得很", r"咋.*", r"啥.*"]

def analyze_features(text):
    """快速特徵分析"""
    mainland_count = sum(1 for term in mainland_terms if term in text)
    pattern_count = sum(1 for pattern in mainland_patterns if re.search(pattern, text))
    return {
        "mainland_terms": [term for term in mainland_terms if term in text],
        "pattern_matches": pattern_count,
        "authenticity_score": mainland_count + pattern_count
    }


async def mainland_score_api_async(text, session, model_endpoint, api_key, model_name):
    """使用你提供的 API 非同步推論大陸用語分數"""

    prompt = f"""
    你是語言檢測工具，請專業的針對下列文本按五項標準打分，每項為 0 或 1，總分為 0~5。

評分標準：
1. 大陸特有詞彙：計算機、軟件、出租車、地鐵等
2. 大陸語法習慣：挺...的、蠻...的、咋樣等  
3. 大陸口語表達：搞定、整、弄等
4. 避免繁體用語：不含電腦、軟體、資料等
5. 整體大陸化程度：綜合評估

下面是要判斷的文本
文本：{text}

請按格式回答：
大陸特有詞彙:0
大陸語法習慣:0
大陸口語表達:0
避免繁體用語:1
整體大陸化程度:0
總分:1"""

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": model_name,
        "messages": [
            {"role": "user", "content": prompt}
        ],
        "temperature": 0.1,
        "max_tokens": 100
    }

    try:
        async with session.post(model_endpoint, headers=headers, json=payload, timeout=60) as response:
            data = await response.json()
            reply = data.get("choices", [{}])[0].get("message", {}).get("content", None)
            return reply
    except Exception as e:
        return str(e)


from tqdm.asyncio import tqdm_asyncio

def parse_scores(reply):
    if not reply or not isinstance(reply, str):
        # API 沒回東西，直接回預設分數
        return {
            "大陸特有詞彙": 0,
            "大陸語法習慣": 0,
            "大陸口語表達": 0,
            "避免繁體用語": 0,
            "整體大陸化程度": 0,
            "總分": 0
        }

    categories = ['大陸特有詞彙', '大陸語法習慣', '大陸口語表達', '避免繁體用語', '整體大陸化程度']
    scores = {}
    for cat in categories:
        match = re.search(fr"{cat}\s*[:：]\s*(\d)", reply)
        if match:
            scores[cat] = int(match.group(1))
        else:
            scores[cat] = 0  # 找不到就補 0
    scores['總分'] = sum(scores.values())
    return scores

async def process_dataset_async_batched(df, model_endpoint, api_key, model_name="gemma-3-27b-it",
                                        text_col='text', sample_size=200, threshold=3, batch_size=20):
    print(f"📊 處理資料集：{len(df)} 筆")
    """
    加入斷點續存的非同步批次處理
    """
    print(f"📊 處理資料集：{len(df)} 筆")
    # 👉 如果你想處理整個 DataFrame，就不要 sample
    if sample_size:
        df = df.sample(n=min(sample_size, len(df)), random_state=42)

    last_index = load_checkpoint()
    results = []
    authentic_texts = []
    generic_texts = []

    # 如果有部分結果，先讀取
    if os.path.exists(PARTIAL_FILE):
        partial_df = pd.read_csv(PARTIAL_FILE)

        # 🔑 加這段：把 CSV 中存成字串的 dict 轉回真正的 dict
        for col in ["features", "scores"]:
            if col in partial_df.columns:
                partial_df[col] = partial_df[col].apply(
                    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
                )

        results = partial_df.to_dict("records")
        print(f"🔁 已讀取部分處理結果：{len(results)} 筆")
        done_indices = set(partial_df["index"].tolist())
    else:
        done_indices = set()

    texts = df[text_col].tolist()
    indices = df.index.tolist()

    async with aiohttp.ClientSession() as session:
        for batch_start in tqdm(range(0, len(texts), batch_size), desc="非同步批次推論"):
            batch_texts = texts[batch_start:batch_start+batch_size]
            batch_indices = indices[batch_start:batch_start+batch_size]

            # 跳過已處理的索引
            filtered_batch = [
                (idx, text) for idx, text in zip(batch_indices, batch_texts)
                if idx not in done_indices and idx > last_index
            ]
            if not filtered_batch:
                continue

            tasks = [
                mainland_score_api_async(text, session, model_endpoint, api_key, model_name)
                for text in batch_texts
            ]
            responses = await asyncio.gather(*tasks)

            for i, response in enumerate(responses):
                text = batch_texts[i]
                idx = batch_indices[i]
                features = analyze_features(text)
                scores = parse_scores(response)

                result = {
                    'index': idx,
                    'text': text,
                    'text_length': len(text),
                    'features': features,
                    'scores': scores,
                    'response': response,
                    'success': scores is not None
                }

                if scores and scores.get("總分", 0) >= threshold:
                    result['category'] = "真正大陸用語"
                    authentic_texts.append(result)
                else:
                    result['category'] = "通用簡體中文"
                    generic_texts.append(result)

                results.append(result)
                done_indices.add(idx)
            
            pd.DataFrame(results).to_csv(PARTIAL_FILE, index=False, encoding="utf-8-sig")
            save_checkpoint(max(done_indices))

    return results, authentic_texts, generic_texts


def save_results(results, authentic_texts, generic_texts):
    """儲存篩選結果 - 支援切分資料格式"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. 完整結果
    full_data = []
    for r in results:
        row = {
            'text': r['text'],
            'text_length': r['text_length'],
            'category': r['category'],
            'success': r['success'],
            'authenticity_score': r['features']['authenticity_score'],
            'mainland_terms': ','.join(r['features']['mainland_terms'])
        }

        
        # 添加切分相關欄位（如果存在）
        original_row = available_data.iloc[r['index']]
        for col in [
            'source_type','source','fragment_length','augmentation_method',
            'original_idx','fragment_index','fragment_start','fragment_end'
        ]:
            if col in original_row:
                row[col] = original_row[col]

        if 'source_type' in original_row:
            row['source_type'] = original_row['source_type']
        if 'source' in original_row:
            row['source'] = original_row['source']
        if 'fragment_length' in original_row:
            row['fragment_length'] = original_row['fragment_length']
        if 'augmentation_method' in original_row:
            row['augmentation_method'] = original_row['augmentation_method']
        
        if r['scores']:
            row.update({f'score_{k}': v for k, v in r['scores'].items()})
        full_data.append(row)
    
    full_df = pd.DataFrame(full_data)
    full_file = f"mainland_filtering_complete_{timestamp}.csv"
    full_df.to_csv(full_file, index=False, encoding='utf-8-sig')
    
    # 2. 高質量大陸用語數據（切分格式）
    if authentic_texts:
        authentic_data = []
        for r in authentic_texts:
            original_row = available_data.iloc[r['index']]
            auth_row = {
                'text': r['text'],
                'total_score': r['scores']['總分'],
                'mainland_terms': ','.join(r['features']['mainland_terms']),
                'text_length': r['text_length']
            }
            
            # 保留切分相關欄位
            if 'source_type' in original_row:
                auth_row['source_type'] = original_row['source_type']
            if 'source' in original_row:
                auth_row['source'] = original_row['source']
            if 'fragment_length' in original_row:
                auth_row['fragment_length'] = original_row['fragment_length']
            if 'augmentation_method' in original_row:
                auth_row['augmentation_method'] = original_row['augmentation_method']
            if 'original_idx' in original_row:
                auth_row['original_idx'] = original_row['original_idx']
            if 'fragment_index' in original_row:
                auth_row['fragment_index'] = original_row['fragment_index']
            
            authentic_data.append(auth_row)
        
        auth_df = pd.DataFrame(authentic_data)
        auth_csv = f"authentic_mainland_texts_{timestamp}.csv"
        auth_json = f"authentic_mainland_texts_{timestamp}.json"
        
        auth_df.to_csv(auth_csv, index=False, encoding='utf-8-sig')
        auth_df.to_json(auth_json, orient='records', force_ascii=False, indent=2)
        
        print(f"💾 儲存完成:")
        print(f"  📄 完整結果: {full_file}")
        print(f"  ✅ 高質量句子片段數據: {auth_csv}")
        print(f"  📋 JSON格式: {auth_json}")
        
        # 顯示切分資料統計
        if 'source' in auth_df.columns:
            print(f"\n📊 高質量數據來源分布:")
            print(auth_df['source'].value_counts())
        
        return full_df, auth_df
    
    return full_df, None

# 主要執行流程
print("="*60)

# 檢查可用資料集 (優先使用最終切分句子片段資料集)
available_data = None
text_column = 'text'

if 'final_split_augmented_df' in locals() and final_split_augmented_df is not None:
    available_data = final_split_augmented_df
    source_name = "最終句子片段擴增資料集"
elif 'split_dataset_df' in locals() and split_dataset_df is not None:
    available_data = split_dataset_df
    source_name = "句子片段資料集"
elif 'optimized_augmented_df' in locals() and optimized_augmented_df is not None:
    available_data = optimized_augmented_df
    source_name = "優化擴增資料集"
elif 'dataset_df' in locals() and dataset_df is not None:
    available_data = dataset_df  
    source_name = "原始資料集"

if available_data is not None:
    print(f"✅ 使用 {source_name}，共 {len(available_data)} 筆記錄")
    
    # 執行篩選（可調整參數）
    MODEL_API_ENDPOINT = "https://labor-openwebui.dgx-coolify.apmic.ai/api/chat/completions"
    OPENWEBUI_API_KEY = API_KEY
    MODEL_NAME = "gemma-3-27b-it"
    SAMPLE_SIZE = 100
    THRESHOLD = 3
    BATCH_SIZE = 20 

    print(f"\n🚀 開始非同步批次處理，每批 {BATCH_SIZE} 筆...")

    # ❗❗❗ 這裡不要用 asyncio.run()，直接 await
    results, authentic_results, generic_results = await process_dataset_async_batched(
        df=available_data,
        model_endpoint=MODEL_API_ENDPOINT,
        api_key=OPENWEBUI_API_KEY,
        model_name=MODEL_NAME,
        text_col=text_column,
        sample_size=None,
        threshold=THRESHOLD,
        batch_size=BATCH_SIZE
    )
    
    # 統計結果
    print(f"\n📊 篩選結果統計:")
    print(f"  ✅ 真正大陸用語: {len(authentic_results)} 筆")
    print(f"  🗑️ 通用簡體中文: {len(generic_results)} 筆")
    print(f"  📈 篩選率: {len(authentic_results)/len(results)*100:.1f}%")
    
    # 顯示範例
    if authentic_results:
        print(f"\n📝 高質量大陸用語範例:")
        for i, r in enumerate(authentic_results[:3]):
            preview = r['text'][:60] + "..." if len(r['text']) > 60 else r['text']
            print(f"  {i+1}. (得分:{r['scores']['總分']}) {preview}")
    
    # 儲存結果
    print(f"\n💾 儲存結果...")
    full_df, auth_df = save_results(results, authentic_results, generic_results)
    
    # 設定全域變數
    globals()['mainland_filtering_results'] = results
    globals()['authentic_mainland_data'] = auth_df
    
    print(f"\n🎉 大陸用語識別與篩選完成！")
    print(f"📋 可用變數: mainland_filtering_results, authentic_mainland_data")
    print(f"🎯 最終輸出為句子級別的片段資料 (10-50字)")
    
else:
    print("❌ 沒有找到可用的資料集")
    print("💡 請先執行前面的資料載入、文本切分或擴增步驟")

print("="*60)

🚀 啟動最終版大陸用語識別系統...
✅ 使用 句子片段資料集，共 36670 筆記錄

🚀 開始非同步批次處理，每批 20 筆...
📊 處理資料集：36670 筆
📊 處理資料集：36670 筆
🔄 偵測到進度檔，將從 index 35803 繼續
🔁 已讀取部分處理結果：100 筆


非同步批次推論: 100%|██████████| 1834/1834 [09:24<00:00,  3.25it/s]


📊 篩選結果統計:
  ✅ 真正大陸用語: 109 筆
  🗑️ 通用簡體中文: 761 筆
  📈 篩選率: 11.2%

📝 高質量大陸用語範例:
  1. (得分:3) 目前从国内来看，instagram略胜一筹，因为国内能买的上iphone的谁不自恋？
  2. (得分:4) 鱼子酱咸鱼什么的：冲着鱼子酱点的， 结果真得不咋地，哎，看来我这辈子也就是个穷人的命啦。
  3. (得分:3) 小黄鱼烧豆腐：很嫩很嫩的，鱼也嫩，豆腐也嫩，就是一碰就碎，有点不知道怎么下手。

💾 儲存結果...
💾 儲存完成:
  📄 完整結果: mainland_filtering_complete_20250911_203642.csv
  ✅ 高質量句子片段數據: authentic_mainland_texts_20250911_203642.csv
  📋 JSON格式: authentic_mainland_texts_20250911_203642.json

🎉 大陸用語識別與篩選完成！
📋 可用變數: mainland_filtering_results, authentic_mainland_data
🎯 最終輸出為句子級別的片段資料 (10-50字)
